# Generate Synthetic Dataset with LLM

Reference: [Fine-Tuning Embeddings for RAG with Synthetic Data](https://medium.com/llamaindex-blog/fine-tuning-embeddings-for-rag-with-synthetic-data-e534409a3971)

Generate a synthetic dataset of (query, relevant documents) pairs from a corpus of **documents without labelers** by leveraging LLM.

## Generate Corpus

In [1]:
import json
import re
import uuid

from llama_index import SimpleDirectoryReader
from llama_index.llms import OpenAI
from llama_index.node_parser import SimpleNodeParser
from llama_index.schema import MetadataMode

# from tqdm.notebook import tqdm
from tqdm import tqdm

In [2]:
TRAIN_FILES = ['afa_docs/merkblatt-fuer-arbeitslose_ba036520.pdf']
VAL_FILES = ['afa_docs/dok_ba035980.pdf']

TRAIN_CORPUS_FPATH = 'afa_docs/train_corpus.json'
VAL_CORPUS_FPATH = 'afa_docs/val_corpus.json'

In [3]:
def load_corpus(files, verbose=False):
    if verbose:
        print(f"Loading files {files}")

    reader = SimpleDirectoryReader(input_files=files)
    docs = reader.load_data()
    if verbose:
        print(f'Loaded {len(docs)} docs')
    
    parser = SimpleNodeParser.from_defaults()
    nodes = parser.get_nodes_from_documents(docs, show_progress=verbose)

    if verbose:
        print(f'Parsed {len(nodes)} nodes')

    corpus = {node.node_id: node.get_content(metadata_mode=MetadataMode.NONE) for node in nodes}
    return corpus

## TODO
RANDOMIZE THE TRAIN/VAL SETS <----------------------------------------------------------

NOW IT USED A PDF FOR TRAIN AND THE OTHER FOR VALIDATION

In [4]:
train_corpus = load_corpus(TRAIN_FILES, verbose=True)
val_corpus = load_corpus(VAL_FILES, verbose=True)

Loading files ['afa_docs/merkblatt-fuer-arbeitslose_ba036520.pdf']
Loaded 103 docs


/home/daniele/Desktop/Projects/DSR_project_ideas/venv_DSR_project_ideas/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Parsing documents into nodes: 100%|██████████| 103/103 [00:00<00:00, 3038.84it/s]


Parsed 103 nodes
Loading files ['afa_docs/dok_ba035980.pdf']
Loaded 40 docs


Parsing documents into nodes: 100%|██████████| 40/40 [00:00<00:00, 3403.09it/s]

Parsed 40 nodes


In [5]:
print(f"Type: {type(train_corpus)}")
print(f"Length: {len(train_corpus)}")
for key in list(train_corpus.keys())[0:2]:
    print(train_corpus[key])
    print("-"*80)

Type: <class 'dict'>
Length: 103
49466_BA_MB_1.indd   1 10.02.2015   13:20:58Agentur für Arbeit  
Musterstadthausen  Merkblatt
1Merkblatt für
Arbeitslose 
Ihre Rechte –
Ihre Pflichten
--------------------------------------------------------------------------------
3 
Ihre Agentur für Arbeit hält eine Fülle von 
 Informationen für Sie bereit. 
Neben den Informationen in diesem Merkblatt finden 
Sie unter » www.arbeitsagentur.de  unser umfassen ­
des Online-Angebot der „eServices “ sowie ein 
 interessantes Informationsangebot aus allen Aufgaben ­
bereichen der Bundesagentur für Arbeit. Sie erhalten 
wertvolle Tipps zu den Themen Ausbil ­
dung, Berufs- und Studienwahl, Weiter ­
bildung, wichtige Informationen über 
Geldleistungen sowie ein umfangreiches 
Serviceangebot.
Über das Job- und Serviceportal  
» www.arbeitsagentur.de  können Sie beispielsweise:
•  sich arbeitsuchend und arbeitslos melden,
•  Geldleistungen, wie Arbeitslosengeld, beantragen
•  Fragen zum Arbeitslosengeld unserem

In [6]:
with open(TRAIN_CORPUS_FPATH, 'w+') as f:
    json.dump(train_corpus, f)

with open(VAL_CORPUS_FPATH, 'w+') as f:
    json.dump(val_corpus, f)

## Generate synthetic queries

Use an LLM (e.g., gpt-3.5-turbo) to generate questions using each text chunk in the corpus as context.

For both training and validation, it creates pairs (`generated question`, `text chunk as context`).These pairs are used as data points in the finetuning dataset.

In [7]:
TRAIN_QUERIES_FPATH = 'afa_docs/train_val_data/train_queries.json'
TRAIN_RELEVANT_DOCS_FPATH = 'afa_docs/train_val_data/train_relevant_docs.json'

VAL_QUERIES_FPATH = 'afa_docs/train_val_data/val_queries.json'
VAL_RELEVANT_DOCS_FPATH = 'afa_docs/train_val_data/val_relevant_docs.json'

In [8]:
with open(TRAIN_CORPUS_FPATH, 'r+') as f:
    train_corpus = json.load(f)

with open(VAL_CORPUS_FPATH, 'r+') as f:
    val_corpus = json.load(f)

In [9]:
def generate_queries(
    corpus,
    num_questions_per_chunk=2,
    prompt_template=None,
    verbose=False,
):
    """
    Automatically generate hypothetical questions that could be answered with
    doc in the corpus.
    """
    llm = OpenAI(model='gpt-3.5-turbo')

    prompt_template = prompt_template or """\
    Context information is below.
    
    ---------------------
    {context_str}
    ---------------------
    
    Given the context information and not prior knowledge.
    generate only questions based on the below query.
    
    You are a Teacher/ Professor. Your task is to setup \
    {num_questions_per_chunk} questions for an upcoming \
    quiz/examination. The questions should be diverse in nature \
    across the document. Restrict the questions to the \
    context information provided."
    """

    queries = {}
    relevant_docs = {}
    # for node_id, text in corpus.items():
    for node_id, text in tqdm(corpus.items()):
        query = prompt_template.format(context_str=text, num_questions_per_chunk=num_questions_per_chunk)
        response = llm.complete(query)
 
        result = str(response).strip().split("\n")
        questions = [
            re.sub(r"^\d+[\).\s]", "", question).strip() for question in result
        ]
        questions = [question for question in questions if len(question) > 0]
        
        for question in questions:
            question_id = str(uuid.uuid4())
            queries[question_id] = question
            relevant_docs[question_id] = [node_id]
    return queries, relevant_docs

In [10]:
train_corpus_small = dict()
i = 0
for key, value in train_corpus.items():
    train_corpus_small[key] = value
    i += 1
    if i > 10:
        break

In [11]:
page = list(train_corpus_small.keys())[6] # a page in the document
print(train_corpus_small[page])

8• Unter Umständen müssen Sie mit dem Wegfall der 
Leistung oder mit Sperrzeiten rechnen, wenn Sie
•  sich nicht selbst aktiv um Arbeit bemühen,
•  die während Ihrer Arbeitslosigkeit von der Agentur 
für Arbeit geforderten Eigenbemühungen nicht 
nachweisen,
•  zumutbare Arbeitsmöglichkeiten nicht nutzen,
•  Eingliederungsmaßnahmen (z.  B. Maßnahmen  
der beruflichen Weiterbildung oder Maßnahmen 
zur Aktivierung und beruflichen Eingliederung)  
ablehnen  
oder
•  einer Aufforderung, sich zu melden oder zu einem 
Untersuchungstermin zu erscheinen, nicht folgen.
• Bitte melden Sie Ihrer zuständigen Agentur für Arbeit 
sofort alle Änderungen, die Ihren Leistungsanspruch 
beeinflussen. Teilen Sie bitte insbesondere umge ­
hend jede Änderung des Familienstandes, der Lohn ­
steuerklasse und des Faktors mit. Wenn Sie mit Ihrer 
Ehegattin / Ihrem Ehegatten oder in einer eingetrage ­
nen Lebenspartnerschaft die Lohnsteuerklassen 
wechseln, lassen Sie sich wegen der  finanziellen 
Auswirkungen un

In [12]:
val_corpus_small = dict()
i = 0
for key, value in val_corpus.items():
    val_corpus_small[key] = value
    i += 1
    if i > 10:
        break

In [13]:
page = list(val_corpus_small.keys())[6] # a page in the document
print(val_corpus_small[page])

7Inhaltsverzeichnis
5 Sie haben in Deutschland  gearbeitet  
und haben als Grenzgängerin bzw.  
Grenzgänger im (benach  barten) Ausland  
gewohnt?  31
5.1 Zusätzliche Arbeitsuchendmeldung von 
 Grenzgängerinnen bzw. Grenzgängern im  
bisherigen Beschäftigungsstaat  31
5.2 Arbeitslosengeld ausnahmsweise  
von Deutschland  32
5.3 Auswirkungen auf Ansprüche der Deutschen 
Rentenversicherung  32
6 Sonderregelungen  34
6.1 Drittstaatsangehörige  34
6.2 Staaten der früheren SFR Jugoslawien  
( außer Slowenien und Kroatien)  35
7 Was Sie sonst noch wissen sollten  37
Anhänge
Anhang 1: Zuständige Stellen  38
Anhang 2: Weitere Merkblätter  39


In [14]:
train_queries, train_relevant_docs = generate_queries(train_corpus_small)

100%|██████████| 11/11 [00:49<00:00,  4.52s/it]


In [15]:
val_queries, val_relevant_docs = generate_queries(val_corpus_small)

100%|██████████| 11/11 [00:45<00:00,  4.11s/it]


In [16]:
with open(TRAIN_QUERIES_FPATH, 'w+') as f:
    json.dump(train_queries, f)

with open(TRAIN_RELEVANT_DOCS_FPATH, 'w+') as f:
    json.dump(train_relevant_docs, f)

with open(VAL_QUERIES_FPATH, 'w+') as f:
    json.dump(val_queries, f)

with open(VAL_RELEVANT_DOCS_FPATH, 'w+') as f:
    json.dump(val_relevant_docs, f)

## Merge data

Reorganize the data for easier accessing the training and evaluation datasets

In [17]:
TRAIN_DATASET_FPATH = 'afa_docs/train_val_data/train_dataset.json'
VAL_DATASET_FPATH = 'afa_docs/train_val_data/val_dataset.json'

In [18]:
train_dataset = {
    'queries': train_queries,
    'corpus': train_corpus,
    'relevant_docs': train_relevant_docs,
}

val_dataset = {
    'queries': val_queries,
    'corpus': val_corpus,
    'relevant_docs': val_relevant_docs,
}

In [19]:
with open(TRAIN_DATASET_FPATH, 'w+') as f:
    json.dump(train_dataset, f)

with open(VAL_DATASET_FPATH, 'w+') as f:
    json.dump(val_dataset, f)